In [1]:
import os, sys
from pathlib import Path
sys.path.append('../src')
sys.path.append('../src/utils/')
from spiking_dataloader import WISDM_spiking_dataloader, WisdmDatasetParser
from inibitory_network.inibitory_network_h1_test import InibitoryLifNet
from lava.proc.io.sink import RingBuffer as SinkRingBuffer
from lava.proc.io.source import RingBuffer as SourceRingBuffer
from lava.proc.monitor.process import Monitor
from output_process import OutputProcess
from lava.proc.monitor.process import Monitor
import matplotlib.pyplot as plt
import numpy as np


In [2]:
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.utils.weightutils import SignMode

path = f"{Path.home()}/snntorch_network/notebook/Trained/network_best.npz"

data = np.load(path,allow_pickle=True)


linear1_w= data['linear1']
leaky1_vth= data['leaky1_vth']
leaky1_betas= 1-data['leaky1_betas'] 
leaky1_betas= leaky1_betas if leaky1_betas >= 0 else np.zeros(leaky1_betas.shape)
print(f"leaky1_betas: {leaky1_betas}")
print(f"leaky1_vth: {leaky1_vth}")
linear2_w = data['linear2']
leaky2_vth= data['recurrent_vth']
leaky2_betas= 1 - data['recurrent_betas']
leaky2_betas= leaky2_betas if  leaky2_betas >= 0 else np.zeros(leaky2_betas.shape)
print(f"leaky2_betas: {leaky2_betas}")
print(f"leaky2_vth: {leaky2_vth}")

recurrent_in_weights = data['input_dense']
recurrent_out_weights = - data['output_dense']
recurrent_vth = data['activation_vth']
recurrent_leaky_betas = 1 - data['activation_betas']
recurrent_leaky_betas= recurrent_leaky_betas if recurrent_leaky_betas >= 0 else np.zeros(recurrent_leaky_betas.shape)
print(f"recurrent_leaky_betas: {recurrent_leaky_betas}")
print(f"recurrent_vth: {recurrent_vth}")

linear3_w = data['linear3']
leaky3_vth= data['leaky2_vth']
leaky3_betas= 1 - data['leaky2_betas']
leaky3_betas= leaky3_betas if leaky3_betas >= 0 else np.zeros(leaky3_betas.shape)
print(f"leaky3_betas: {leaky3_betas}")
print(f"leaky3_vth: {leaky3_vth}")


    

linear1 = Dense(weights=linear1_w, num_message_bits=32, name="linear1")

leaky1 = LIF(shape=(linear1_w.shape[0],),
                    u = np.zeros(linear1_w.shape[0]),
                    v = np.zeros(linear1_w.shape[0]),
                    du = 1.0,
                    dv = leaky1_betas,
                    vth=leaky1_vth,
                    log_config=0,
                    name= "leaky1"
                )
linear1.a_out.connect(leaky1.a_in)
linear2 = Dense(weights=linear2_w, num_message_bits=0, sign_mode=SignMode.MIXED, name="linear2")
linear2.s_in.connect_from(leaky1.s_out)

leaky2 = LIF(shape=(linear2_w.shape[0],),
                    u = np.zeros(linear2_w.shape[0]),
                    v = np.zeros(linear2_w.shape[0]),
                    du = 1.0,
                    dv= leaky2_betas,
                    vth=leaky2_vth,
                    log_config=0,
                    name= "leaky2"
                )
#sum.a_out.connect(leaky2.a_in)
linear2.a_out.connect(leaky2.a_in)
#leaky2.a_in.connect_from(linear2.a_out)

recurrent_in = Dense(weights=recurrent_in_weights, num_message_bits=0, name="recurrent_in")
leaky2.s_out.connect(recurrent_in.s_in)

ahpc = LIF(shape=(recurrent_in_weights.shape[0],),
                    u = np.zeros(recurrent_in_weights.shape[0]),
                    v = np.zeros(recurrent_in_weights.shape[0]),
                    du = 1.0,
                    dv = recurrent_leaky_betas,
                    vth=recurrent_vth,
                    log_config=0,
                    name= "inibitory_leaky"
                )

recurrent_in.a_out.connect(ahpc.a_in)

recurrent_out = Dense(weights=recurrent_out_weights, num_message_bits=0, name="recurrent_out")
recurrent_out.s_in.connect_from(ahpc.s_out)
recurrent_out.a_out.connect(leaky2.a_in)

linear3 = Dense(weights=linear3_w, num_message_bits=0, name="linear3")
linear3.s_in.connect_from(leaky2.s_out)

leaky3 = LIF(shape=(linear3_w.shape[0],),
                    u = np.zeros(linear3_w.shape[0]),
                    v = np.zeros(linear3_w.shape[0]),
                    du = 1.0,
                    dv = leaky3_betas,
                    vth=leaky3_vth,
                    log_config=0,
                    name= "leaky3"
                )
leaky3.a_in.connect_from(linear3.a_out)

leaky1_betas: 0.6422713398933411
leaky1_vth: 1.6034977436065674
leaky2_betas: 0.1867157220840454
leaky2_vth: 0.8977742195129395
recurrent_leaky_betas: 0.0
recurrent_vth: 0.7044668197631836
leaky3_betas: 0.0
leaky3_vth: 0.8803743720054626


In [3]:

from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg, Loihi2SimCfg
from lava.utils.float2fixed import Float2FixedConverter
import numpy as np
#from tqdm import tqdm

converter = Float2FixedConverter()
converter.set_run_cfg(fixed_pt_run_cfg=Loihi1SimCfg(select_tag='fixed_pt'),
                      floating_pt_run_cfg=Loihi1SimCfg(select_tag='floating_pt'))
converter.convert([leaky2, leaky3, ahpc, linear2, recurrent_in, recurrent_out, linear3], num_steps=200)





In [4]:
converter.scaled_params


{3: {'bias_exp': 0,
  'vth': 25745,
  'u': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.utils.weightutils import SignMode   
from lif_mod import LIFEncoder
from dense_mod import DenseEncoder

linear1 = DenseEncoder(weights=linear1_w, num_message_bits=32, name="linear1")

leaky1 = LIFEncoder(shape=(linear1_w.shape[0],),
                    u = np.zeros(linear1_w.shape[0]),
                    v = np.zeros(linear1_w.shape[0]),
                    du = 1.0,
                    dv = leaky1_betas,
                    vth=leaky1_vth,
                    log_config=0,
                    name= "leaky1"
                )
linear1.a_out.connect(leaky1.a_in)
linear2 = Dense(**converter.scaled_params[linear2.id],
                sign_mode=SignMode.MIXED, name="linear2")

linear2.s_in.connect_from(leaky1.s_out)

leaky2 = LIF(shape=(linear2_w.shape[0],),
                    **converter.scaled_params[leaky2.id],
                    name= "leaky2"
                )
#sum.a_out.connect(leaky2.a_in)
linear2.a_out.connect(leaky2.a_in)
#leaky2.a_in.connect_from(linear2.a_out)

recurrent_in = Dense(**converter.scaled_params[recurrent_in.id],
                     name="recurrent_in")

leaky2.s_out.connect(recurrent_in.s_in)

ahpc = LIF(shape=(recurrent_in_weights.shape[0],),
                    **converter.scaled_params[ahpc.id],
                    log_config=0,
                    name= "inibitory_leaky"
                )

recurrent_in.a_out.connect(ahpc.a_in)

recurrent_out = Dense(**converter.scaled_params[recurrent_out.id],
                       name="recurrent_out")
recurrent_out.s_in.connect_from(ahpc.s_out)
recurrent_out.a_out.connect(leaky2.a_in)

linear3 = Dense(**converter.scaled_params[linear3.id],
                name="linear3")

linear3.s_in.connect_from(leaky2.s_out)

leaky3 = LIF(shape=(linear3_w.shape[0],),
                    **converter.scaled_params[leaky3.id],
                    log_config=0,
                    name= "leaky3"
                )

leaky3.a_in.connect_from(linear3.a_out)



In [6]:
num_samples = 100 
signal_step = 40
clear_intervall = 40
time_steps = signal_step + clear_intervall

dataset = WisdmDatasetParser('../data/data_watch_subset_0_40.npz', norm=None, class_sublset='custom', subset_list=[0, 4, 6, 8, 9, 10, 14])
#val_set = dataset.get_validation_set(shuffle=False, subset=num_samples)
val_set = dataset.get_validation_set()
val_set[0].shape
num_samples = val_set[0].shape[0]
spiking_loader = WISDM_spiking_dataloader(val_set ,clear_intervall=clear_intervall)
out_sink = OutputProcess(7,num_samples,time_steps, 0)

(6,)
(6,)
ytrain shape (55404, 18)
yval shape (18468, 18)
ytest shape (18469, 18)
num classes train dataset: 7 occurrences of each class:[3127 3044 3102 3047 3150 3087 2973]
num classes eval dataset: 7 occurrences of each class:[1035 1048 1122  996 1110 1053 1007]
num classes test dataset: 7 occurrences of each class:[1046 1048 1046 1036 1076 1026  982]


In [7]:
spiking_loader.data_out.connect(linear1.s_in)
leaky3.s_out.connect(out_sink.spikes_in)
out_sink.label_in.connect_from(spiking_loader.label_out)

In [8]:

from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg, Loihi2SimCfg
import numpy as np
from tqdm import tqdm

for i in tqdm(range(num_samples)):
      out_sink.run(condition=RunSteps(num_steps=time_steps),
                  run_cfg=Loihi1SimCfg(select_sub_proc_model=True,
                  select_tag='fixed_pt'))
      leaky1.v.set(np.zeros(leaky1.v.shape))
      leaky1.u.set(np.zeros(leaky1.u.shape))
      leaky2.v.set(np.zeros(leaky2.v.shape))
      leaky2.u.set(np.zeros(leaky2.u.shape))
      leaky3.v.set(np.zeros(leaky3.v.shape))
      leaky3.u.set(np.zeros(leaky3.u.shape))
      ahpc.v.set(np.zeros(ahpc.v.shape))
      ahpc.u.set(np.zeros(ahpc.u.shape))
      linear1.a_buff.set(np.zeros(linear1.a_buff.shape))
      linear2.a_buff.set(np.zeros(linear2.a_buff.shape))
      linear3.a_buff.set(np.zeros(linear3.a_buff.shape))
        
ground_truth = val_set[1][:num_samples]
predictions = out_sink.pred_labels.get().astype(int)
# Stop the execution
out_sink.stop()

accuracy = np.sum(ground_truth==predictions)/ground_truth.size * 100

print(f"\nGround truth: {ground_truth}\n"
      f"Predictions : {predictions}\n"
      f"Accuracy    : {accuracy}")

  0%|          | 0/7371 [00:00<?, ?it/s]/home/franzhd/lava/src/lava/magma/compiler/compiler_graphs.py:868: UserWarning: Cannot import module '<module 'dataloader' from '/home/franzhd/lava_AHPC/experiments/../src/utils/dataloader.py'>' when searching ProcessModels for Process 'OutputProcess'.
  warnings.warn(
/home/franzhd/lava/src/lava/magma/compiler/compiler_graphs.py:868: UserWarning: Cannot import module '<module 'dataloader' from '/home/franzhd/lava_AHPC/experiments/../src/utils/dataloader.py'>' when searching ProcessModels for Process 'WISDM_spiking_dataloader'.
  warnings.warn(
100%|██████████| 7371/7371 [13:26<00:00,  9.14it/s]



Ground truth: [2 5 4 ... 5 5 6]
Predictions : [2 5 4 ... 5 5 6]
Accuracy    : 81.08804775471442
